<a href="https://colab.research.google.com/github/RoniAndarsyah/LSTM_NaiveBayesMultinomial/blob/master/LSTM_dan_NaiveBayesM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 4.4 MB/s 


Cari Datanya disini saya simpan di Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
import time
import datetime
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns
import string, re
import itertools
import plotly.offline as py
import plotly.graph_objs as go
from collections import Counter

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from textblob import Word

# from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, roc_curve, auc,accuracy_score,f1_score, classification_report
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, confusion_matrix, recall_score, precision_score


from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten,BatchNormalization
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D,Conv1D, MaxPooling1D
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from keras.models import model_from_json
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, Dense, GRU, Dropout, Bidirectional, SpatialDropout1D
from tensorflow.keras.utils import to_categorical

py.init_notebook_mode(connected=True)

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

tf.random.set_seed(1234)

In [6]:
import os
import numpy as np 
import pandas as pd 

Pastikan lokasinya sesuai ya, saya simpan d Folder LSTM pada GDRIVE silahkan ganti sesuai kebutuhan

In [7]:
df=pd.read_csv('drive/MyDrive/LSTM/DATASET-excel.csv', encoding= 'unicode_escape')

In [8]:
df.head()

,Komentar,Kategori,Unnamed: 2,Unnamed: 3
0,"""Kaka tidur yaa, udah pagi, gaboleh capek2""",Non-bullying,NaN,NaN
1,"""makan nasi padang aja begini badannya""",Non-bullying,NaN,NaN
2,"""yang aku suka dari dia adalah selalu cukur je...",Bullying,NaN,NaN
3,"""Hai kak Isyana aku ngefans banget sama kak Is...",Non-bullying,NaN,NaN
4,"""Manusia apa bidadari sih herann deh cantik te...",Non-bullying,NaN,NaN


In [9]:
df.drop(['Unnamed: 2', 'Unnamed: 3'], inplace=True, axis=1)

In [10]:
df.shape

(650, 2)

In [11]:
df.Kategori.value_counts()

Non-bullying    325
Bullying        325
Name: Kategori, dtype: int64

Text Preprocessing: Stemming, Stop-Words removal and Lemmatization

In [12]:
# function to clean the word in html tags
def cleanhtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr,' ',sentence)
    return cleantext

# function to clean the word of any punctuation
def cleanpunc(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#]', r'', sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]', r' ',cleaned)
    return cleaned

def words(text): return re.findall(r'\w+', text.lower())

In [21]:
from nltk.corpus import stopwords


In [14]:
def preprocess_data(data):
    i=0
    str1= ' '
    final_string = []
    for sent in data['Komentar'].values:
        filtered_sentence = []
        sent = cleanhtml(sent)
        # remove html tags
        for w in sent.split():
            for cleaned_words in cleanpunc(w).split():
                if ((cleaned_words.isalpha()) & (len(cleaned_words)>2)):
                    if (cleaned_words.lower() not in stopword):
                        s = (ps.stem(cleaned_words.lower()))
                        filtered_sentence.append(s)
                    else:
                        continue
                else:
                    continue
        str1 = " ".join(filtered_sentence) # final string of cleaned words

        final_string.append(str1)
        i+=1
    return final_string

In [16]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [17]:
def plot_cloud(wordcloud):
    # Set figure size
    plt.figure(figsize=(20, 10))
    # Display image
    plt.imshow(wordcloud) 
    # No axis details
    plt.axis("off")